# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 11:15AM,253825,19,8955486,"Emersa Waterbox, LLC",Released
1,Dec 27 2022 11:00AM,253824,19,ADV80009612,"AdvaGen Pharma, Ltd",Released
2,Dec 27 2022 11:00AM,253824,19,ADV80009613,"AdvaGen Pharma, Ltd",Released
3,Dec 27 2022 11:00AM,253824,19,ADV80009614,"AdvaGen Pharma, Ltd",Released
4,Dec 27 2022 11:00AM,253824,19,ADV80009617,"AdvaGen Pharma, Ltd",Released
5,Dec 27 2022 11:00AM,253824,19,ADV80009618,"AdvaGen Pharma, Ltd",Released
6,Dec 27 2022 11:00AM,253824,19,ADV80009620,"AdvaGen Pharma, Ltd",Released
7,Dec 27 2022 11:00AM,253824,19,ADV80009621,"AdvaGen Pharma, Ltd",Released
8,Dec 27 2022 11:00AM,253824,19,ADV80009622,"AdvaGen Pharma, Ltd",Released
9,Dec 27 2022 11:00AM,253824,19,ADV80009706,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,253822,Executing,13
29,253822,Released,12
30,253823,Released,5
31,253824,Released,17
32,253825,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253821,NaN,1.0
253822,13.0,12.0
253823,NaN,5.0
253824,NaN,17.0
253825,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253783,0.0,1.0
253784,28.0,1.0
253788,0.0,1.0
253789,0.0,1.0
253790,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253783,0,1
253784,28,1
253788,0,1
253789,0,1
253790,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253783,0,1
1,253784,28,1
2,253788,0,1
3,253789,0,1
4,253790,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253783,,1
1,253784,28,1
2,253788,,1
3,253789,,1
4,253790,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC"
1,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd"
18,Dec 27 2022 10:51AM,253822,15,"Brookfield Pharmaceuticals, LLC"
43,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc."
48,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC"
49,Dec 27 2022 10:33AM,253820,10,Emerginnova
52,Dec 27 2022 10:32AM,253819,10,Emerginnova
54,Dec 27 2022 10:15AM,253815,10,ISDIN Corporation
55,Dec 27 2022 10:15AM,253814,10,ISDIN Corporation
108,Dec 27 2022 10:15AM,253813,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC",,1
1,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd",,17
2,Dec 27 2022 10:51AM,253822,15,"Brookfield Pharmaceuticals, LLC",13,12
3,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",,5
4,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",,1
5,Dec 27 2022 10:33AM,253820,10,Emerginnova,,3
6,Dec 27 2022 10:32AM,253819,10,Emerginnova,,2
7,Dec 27 2022 10:15AM,253815,10,ISDIN Corporation,,1
8,Dec 27 2022 10:15AM,253814,10,ISDIN Corporation,,53
9,Dec 27 2022 10:15AM,253813,10,ISDIN Corporation,,66


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC",1,
1,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd",17,
2,Dec 27 2022 10:51AM,253822,15,"Brookfield Pharmaceuticals, LLC",12,13
3,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5,
4,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1,
5,Dec 27 2022 10:33AM,253820,10,Emerginnova,3,
6,Dec 27 2022 10:32AM,253819,10,Emerginnova,2,
7,Dec 27 2022 10:15AM,253815,10,ISDIN Corporation,1,
8,Dec 27 2022 10:15AM,253814,10,ISDIN Corporation,53,
9,Dec 27 2022 10:15AM,253813,10,ISDIN Corporation,66,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC",1,
1,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd",17,
2,Dec 27 2022 10:51AM,253822,15,"Brookfield Pharmaceuticals, LLC",12,13
3,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5,
4,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC",1.0,NaN
1,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd",17.0,NaN
2,Dec 27 2022 10:51AM,253822,15,"Brookfield Pharmaceuticals, LLC",12.0,13.0
3,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5.0,NaN
4,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC",1.0,0.0
1,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd",17.0,0.0
2,Dec 27 2022 10:51AM,253822,15,"Brookfield Pharmaceuticals, LLC",12.0,13.0
3,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5.0,0.0
4,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5583724,421.0,0.0
12,253797,0.0,1.0
15,507645,17.0,13.0
16,253806,1.0,0.0
19,1015237,20.0,0.0
20,253784,1.0,28.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5583724,421.0,0.0
1,12,253797,0.0,1.0
2,15,507645,17.0,13.0
3,16,253806,1.0,0.0
4,19,1015237,20.0,0.0
5,20,253784,1.0,28.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,421.0,0.0
1,12,0.0,1.0
2,15,17.0,13.0
3,16,1.0,0.0
4,19,20.0,0.0
5,20,1.0,28.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,421.0
1,12,Released,0.0
2,15,Released,17.0
3,16,Released,1.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,0.0,1.0,13.0,0.0,0.0,28.0
Released,421.0,0.0,17.0,1.0,20.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,0.0,1.0,13.0,0.0,0.0,28.0
1,Released,421.0,0.0,17.0,1.0,20.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,0.0,1.0,13.0,0.0,0.0,28.0
1,Released,421.0,0.0,17.0,1.0,20.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()